# Portfolio's Building #

### Rebalancing Weights ###

In [1]:
# Import Libraries

# Data Management
import pandas as pd
import numpy as np

# Optimization
from scipy.optimize import minimize

# Visualization
import matplotlib.pyplot as plt

# Handle Files
import sys
import os

# Import Local Functions
sys.path.append(os.path.abspath("../source"))
from config import get_tickers
from data_downloader import get_market_data
from portfolios_helper import calculate_analytics
from markowitz_portfolios_toolkit import markowitz_weights

In [2]:
tickers = get_tickers(mod="2.4")

tickers

In [3]:
# Import data
df_returns = pd.DataFrame()

for ticker in tickers:
    df = get_market_data(
        ticker=ticker, 
        start_date='2013-01-01', 
        end_date='2025-01-01', 
        returns=True
    )
    
    returns_series = df['returns'].rename(ticker)
    
    df_returns = pd.concat([df_returns, returns_series], axis=1)
    
    print(f'Data Ready for {ticker}')

df_returns.index.name = 'date'
df_returns.index = pd.to_datetime(df_returns.index)

In [4]:
df_returns

In [5]:
df_returns.mean()

In [6]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(df_returns.cumsum(), label=df_returns.columns, alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time Index')
plt.ylabel('Cumulative Returns')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [7]:
# Let us Calculate the Weights
def rolling_weights(
    returns, 
    desired_returns, 
    window=252, 
    rebalance_freq=126
):

    # Lists to Store Things
    weights_list = []
    dates = []

    for i in range(window, len(returns), rebalance_freq):
        past_returns = returns.iloc[i - window:i]  # Rolling Window
        past_excepted_returns = past_returns.mean()
        past_cov_matrix = past_returns.cov()

        # Calculate Weights
        w = markowitz_weights(past_excepted_returns, past_cov_matrix, desired_returns)

        # Save weights and dates
        weights_list.append(w)
        dates.append(returns.index[i])

    # Create the DataFrame
    weights_df = pd.DataFrame(weights_list, index=dates, columns=returns.columns)

    # Expand the DataFrame
    weights_df = weights_df.reindex(returns.index, method='ffill')

    return weights_df.dropna()

In [8]:
# Create the DataFrames of Returns
df_weights = rolling_weights(df_returns, 0.001)

df_weights

In [9]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(df_weights, label=df_weights.columns, alpha=1)
# Config
plt.title('Weights Time Series')
plt.xlabel('Time Index')
plt.ylabel('Weights')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [10]:
# Common Index
common_index = df_returns.index.intersection(df_weights.index)
df_returns_reindex = df_returns.reindex(common_index)
df_weights = df_weights.reindex(common_index)

df_returns_reindex

In [11]:
# Create the Portfolio Returns
df_weighted_returns = df_returns_reindex * df_weights

df_weighted_returns

In [12]:
# Create the Portfolio Returns
df_returns_portfolio = df_returns.copy()

# Add the columns
df_returns_portfolio['Portfolio'] =  df_weighted_returns.sum(axis = 1)

df_returns_portfolio['Portfolio'].dropna()

In [13]:
# Time Series Graphs
df_plot = df_returns_portfolio.dropna()

plt.figure(figsize=(10, 6))
plt.plot(df_plot.cumsum(), label=df_plot.columns, alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time Index')
plt.ylabel('Cumulative Returns')
plt.legend()

# Show
plt.grid(True)
plt.show()

### Define different rebalancing frequencies ###

In [14]:
# Desired Return
dr = 0.001

In [15]:
# Create weights for different re-balancing frequencies
df_weights_5d = rolling_weights(df_returns, dr, rebalance_freq=5)
df_weights_21d = rolling_weights(df_returns, dr, rebalance_freq=21)
df_weights_63d = rolling_weights(df_returns, dr, rebalance_freq=63)
df_weights_126d = rolling_weights(df_returns, dr, rebalance_freq=126)
df_weights_252d = rolling_weights(df_returns, dr, rebalance_freq=252)

In [16]:
# Create the Returns
df_weighted_returns_5d = df_returns * df_weights_5d
df_weighted_returns_21d = df_returns * df_weights_21d
df_weighted_returns_63d = df_returns * df_weights_63d
df_weighted_returns_126d = df_returns * df_weights_126d
df_weighted_returns_252d = df_returns * df_weights_252d

In [17]:
# Add the columns
portfolios_df = pd.DataFrame(index = df_returns.loc['2016':].index)

portfolios_df['5d_port'] = df_weighted_returns_5d.sum(axis = 1)
portfolios_df['21d_port'] = df_weighted_returns_21d.sum(axis = 1)
portfolios_df['63d_port'] = df_weighted_returns_63d.sum(axis = 1)
portfolios_df['126d_port'] = df_weighted_returns_126d.sum(axis = 1)
portfolios_df['252d_port'] = df_weighted_returns_252d.sum(axis = 1)

portfolios_df

In [18]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(portfolios_df.cumsum().mul(100), label=portfolios_df.columns, alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Cumulative Returns (%)')
plt.legend()

# Show
plt.grid(True)
plt.show()

### Comparing Portfolios ###

In [19]:
# Now the table
analytics_table = calculate_analytics(portfolios_df)

analytics_table

### Constraining Weights ###

The problem is that we cannot create large portfolios (with many components) using this methodology because, sometimes, many assets will have high levels of leverage. Therefore, we can use other methods:

In [61]:
# Portfolio Variance
def portfolio_variance(w, covariance_matrix):
    return w.T @ covariance_matrix @ w


# Constraint: Sum of weights equal one
def constraint_sum_weights(w):
    return np.sum(w) - 1


# Constraint: Returns equation
def constraint_target_return(w, expected_returns, target_return):
    return w.T @ expected_returns - target_return


# Optimization
def markowitz_optimization(
        expected_returns,
        covariance_matrix,
        target_return
):
    # Initial Weights
    n = len(expected_returns)
    w0 = np.ones(n) / n

    # Set constraints
    constraints = [
        {'type': 'eq', 'fun': constraint_sum_weights},
        {'type': 'eq', 'fun': lambda w: constraint_target_return(w, expected_returns, target_return)}
    ]

    # Set bounds (-1 if allow shorting, 0 if not)
    bounds = [(-1, 1) for _ in range(n)]

    result = minimize(
        portfolio_variance,
        w0,
        args=(covariance_matrix,),
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )

    if result.success:
        return result.x
    else:
        raise ValueError("Optimization failed: " + result.message)

In [73]:
# Import Data
universe_returns = pd.read_csv(r'..\additional_data\stocks_returns.csv')
universe_returns = universe_returns.rename(columns={'Unnamed: 0':'Date'})
universe_returns.set_index('Date', inplace=True)
universe_returns.index = pd.to_datetime(universe_returns.index)
universe_returns = universe_returns.dropna(axis=1)

# Use data from 2015
universe_returns = universe_returns.loc['2015':]

universe_returns

In [84]:
# Components:
mean_returns = universe_returns.loc['2015'].mean()
cov_matrix = universe_returns.loc['2015'].cov()

In [138]:
unrestricted_weights = markowitz_weights(mean_returns, cov_matrix, 0.0003)
unrestricted_weights = pd.Series(unrestricted_weights, index = universe_returns.columns)
unrestricted_weights.name = 'weights'
unrestricted_weights = unrestricted_weights/1000

unrestricted_weights

In [139]:
restricted_weights = markowitz_optimization(mean_returns, cov_matrix, 0.0003)
restricted_weights = pd.Series(restricted_weights, index = universe_returns.columns)
restricted_weights.name = 'weights'

restricted_weights

In [140]:
unrestricted_portfolio = universe_returns.loc['2016':] @ unrestricted_weights
unrestricted_portfolio.name = 'unrestricted_portfolio'

unrestricted_portfolio

In [141]:
restricted_portfolio = universe_returns.loc['2016':] @ restricted_weights
restricted_portfolio.name = 'restricted_portfolio'

restricted_portfolio

In [142]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(unrestricted_portfolio.cumsum().mul(100), label='Portfolios with unlimited weights', alpha=1)
plt.plot(restricted_portfolio.cumsum().mul(100), label='Portfolios with limited weights', alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Cumulative Returns (%)')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [143]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(restricted_portfolio.cumsum().mul(100), label='Portfolios with limited weights', alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Cumulative Returns (%)')
plt.legend()

# Show
plt.grid(True)
plt.show()

In [144]:
# Create the analytics
limit_portfolios = pd.DataFrame(index=restricted_portfolio.index)

limit_portfolios['unrestricted_portfolio'] = unrestricted_portfolio
limit_portfolios['restricted_portfolio'] = restricted_portfolio

limit_portfolios

In [145]:
calculate_analytics(limit_portfolios)

### Compare with a Portfolio with Rebalancing and No Survivorship Bias ###

In [148]:
# Import Data
real_portfolio = pd.read_csv(r'..\additional_data\markowitz_portfolio.csv')
real_portfolio = real_portfolio.rename(columns={'Unnamed: 0':'Date'})
real_portfolio.set_index('Date', inplace=True)
real_portfolio.index = pd.to_datetime(real_portfolio.index)

# Use data from 2016
real_portfolio = real_portfolio.loc['2016':]

real_portfolio

In [149]:
# Time Series Graphs
plt.figure(figsize=(10, 6))
plt.plot(restricted_portfolio.cumsum().mul(100), label='Example Portfolio', alpha=1)
plt.plot(real_portfolio.cumsum().mul(100), label='Realistic Portfolio', alpha=1)

# Config
plt.title('Cumulative Returns Time Series')
plt.xlabel('Time')
plt.ylabel('Cumulative Returns (%)')
plt.legend()

# Show
plt.grid(True)
plt.show()

The difference in returns can be explained by the survivorship bias (and the re-balancing factor)